In [4]:
import json

# Load vibes list
with open("vibeslist.json") as f:
    vibes_list = json.load(f)  # json to python format

# Rule-based vibe keywords (expand as needed)
vibe_keywords = {
    "Coquette": ["lace", "pink", "feminine", "bow", "girly"],
    "Clean Girl": ["slick hair", "minimal", "glowy", "dewy", "simple"],
    "Cottagecore": ["floral", "vintage", "meadow", "farm", "nature", "cottage"],
    "Streetcore": ["urban", "graffiti", "sneakers", "oversized", "streetwear"],
    "Y2K": ["retro", "2000s", "butterfly clips", "low rise", "chrome", "tech"],
    "Boho": ["fringe", "hippie", "festival", "earthy", "loose", "natural"],
    "Party Glam": ["sparkle", "glitter", "high heels", "makeup", "night out"]
}

In [5]:
import spacy
from collections import Counter

nlp = spacy.load("en_core_web_sm")  # en_core_web_sm -> spaCy English Model

def rule_based_vibes(text):
    doc = nlp(text.lower())
    counts = Counter()

    for vibe, keywords in vibe_keywords.items():
        for token in doc:
            for kw in keywords:
                if kw in token.text:
                    counts[vibe] += 1
    return counts

In [6]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('all-MiniLM-L6-v2')  # a small, efficient transformer that generates sentence embeddings.

# Precompute vibe embeddings
vibe_embeddings = model.encode(vibes_list, convert_to_tensor=True)

def ml_based_vibes(text):
    text_embedding = model.encode(text, convert_to_tensor=True)
    cos_scores = util.pytorch_cos_sim(text_embedding, vibe_embeddings)[0]

    scores = {vibe: float(cos_scores[i]) for i, vibe in enumerate(vibes_list)}
    return scores

In [7]:
def classify_vibes(text, rule_weight=0.6, ml_weight=0.4):
    rule_scores = rule_based_vibes(text)
    ml_scores = ml_based_vibes(text)

    final_scores = {}

    for vibe in vibes_list:
        rule_score = rule_scores.get(vibe, 0)
        ml_score = ml_scores.get(vibe, 0.0)
        # Combine using weights
        final_scores[vibe] = rule_weight * rule_score + ml_weight * ml_score

    # Get top 3 vibes
    sorted_vibes = sorted(final_scores.items(), key=lambda x: x[1], reverse=True)
    return [v for v, s in sorted_vibes[:3] if s > 0]

In [11]:
caption = """Sunkissed Summer has landed.
Think golden hour, every hour.

Easy silhouettes. Bare shoulders. Dresses that breathe.

Made for holiday glow, even if you’re just stepping out for coffee."""
hashtags = ""

text_input = caption + " " + hashtags  # or add transcript if available

predicted_vibes = classify_vibes(text_input)
print("Predicted Vibes:", predicted_vibes)

Predicted Vibes: ['Cottagecore', 'Clean Girl', 'Boho']
